In [8]:
from sklearn.metrics import mutual_info_score
#from sklearn.feature_selection import chi2
from scipy.stats import chi2_contingency
from scipy.stats import chi2
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import gaussian_kde, pearsonr
import seaborn as sns
import pandas as pd
rng = np.random.default_rng()

# Mathematical Underpinnings - Lab 4

Tests to verify hipoteses of independence (from Lab 4):

In [9]:
def indep_test_asymptotic(X, Y, stat):

    if stat == "mi":

        stat_value = 2*len(X)*mutual_info_score(X, Y)

    if stat == "chi2":

        test_res = (chi2_contingency(pd.crosstab(X, Y)))
        stat_value = test_res.statistic

    df = (len(np.unique(X)) - 1)*(len(np.unique(Y)) - 1)

    p_value = 1 - chi2.cdf(stat_value, df=df)

    return stat_value, p_value

def indep_test_permutation(X, Y, B, stat="mi"):

    stat_value = mutual_info_score(X, Y)

    condition_p_value = 0
    for b in range(B):
        X_b = np.random.permutation(X)

        stat_value_b = mutual_info_score(X_b, Y)

        if stat_value <= stat_value_b:
            condition_p_value += 1

    p_value = (1 + condition_p_value)/(1 + B)

    return 2*len(X)*stat_value, p_value

## Task 1

In [10]:
# a function which computes CMI
def CMI(X, Y, Z):
    results = []
    for z in np.unique(Z):
        indx = np.where(Z == z)[0]
        prob_z = len(indx)/len(Z)
        results.append(mutual_info_score(X[indx], Y[indx])*prob_z)
    return np.sum(results)

### a)

In [11]:
# CI test based on CMI and asymptotics
def ACI(X, Y, Z):
    cmi = CMI(X, Y, Z)
    stat = 2*len(X)*cmi
    df = (len(np.unique(X))-1)*(len(np.unique(Y))-1)*len(np.unique(Z))
    return stat, 1-chi2.cdf(stat, df=df)

### b)

In [12]:
# CI test based on CMI and permutations
def CPT(X, Y, Z, B=1000):
    cmi = CMI(X, Y, Z)
    results = []
    for i in range(B):
        Xs = []
        for z in np.unique(Z):
            indx = np.where(Z == z)[0]
            Xs.append(rng.permutation(X[indx]))
        X_perm = np.concatenate(Xs)
        cmi_perm = CMI(X_perm, Y, Z)
        results.append(cmi<=cmi_perm)
    p_value = (1+np.sum(results))/(1+B)
    return cmi, p_value

### c)

conditionaly independent

In [20]:
N = 10000

Z = rng.binomial(1, 0.5, N)
X = np.where((rng.normal(0, 1, N) + Z/2)>=0, 1, -1)
Y = np.where((rng.normal(0, 1, N) - Z/2)>=0, 1, -1)

print(ACI(X, Y, Z))
print(CPT(X, Y, Z))

(2.4276046040200416, 0.2970655961791202)
(0.00012138023020100208, 0.3156843156843157)


conditionaly dependent

In [21]:
X = np.where(rng.normal(0, 1, N)>=0, 1, 0)
Z = np.where(rng.normal(0, 1, N)>=0, 1, 0)

Y = np.logical_xor(X, Z).astype(int)

print(ACI(X, Y, Z))
print(CPT(X, Y, Z))

(13862.889364156525, 0.0)
(0.6931444682078263, 0.000999000999000999)


## Task 2

In [22]:
def discretize(data):
    data[data>0] = 1
    data[data<0] = -1
    return data

def sample_from_model1(n=1000):
    Z = discretize(rng.normal(loc=0, scale=1, size=n))
    X = discretize(rng.normal(loc=Z/2, scale=1, size = n))
    Y = discretize(rng.normal(loc=Z/2, scale=1, size=n))
    return X, Y, Z

def sample_from_model2(n=1000):
    X = discretize(rng.normal(loc=0, scale=1, size = n))
    Z = discretize(rng.normal(loc=X/2, scale=1, size=n))
    Y = discretize(rng.normal(loc=Z/2, scale=1, size=n))
    return X, Y, Z

def sample_from_model3(n=1000):
    X = discretize(rng.normal(loc=0, scale=1, size = n))
    Y = discretize(rng.normal(loc=0, scale=1, size=n))
    Z = discretize(rng.normal(loc=(X+Y)/2, scale=1, size=n))
    return X, Y, Z

### a)

    answer:
    conditionally independent by Z, dependent
    conditionally independent by Z, dependent
    conditionally dependent by Z, independent

### b)

In [24]:
X1, Y1, Z1 = sample_from_model1()
X2, Y2, Z2 = sample_from_model2()
X3, Y3, Z3 = sample_from_model3()


print("M1 MI(X, Y)=", mutual_info_score(X1, Y1))
print("M2 MI(X, Y)=", mutual_info_score(X2, Y2))
print("M3 MI(X, Y)=", mutual_info_score(X3, Y3))

print("M1 CMI(X, Y|Z)=", CMI(X1, Y1, Z1))
print("M2 CMI(X, Y|Z)=", CMI(X2, Y2, Z2))
print("M3 CMI(X, Y|Z)=", CMI(X3, Y3, Z3))

M1 MI(X, Y)= 0.01217103182580026
M2 MI(X, Y)= 0.013161847159061435
M3 MI(X, Y)= 1.8097470140754357e-05
M1 CMI(X, Y|Z)= 0.0006968093630406963
M2 CMI(X, Y|Z)= 0.0008915689687091646
M3 CMI(X, Y|Z)= 0.01031234541755329


### c)

In [25]:
print("M1 permutation ", CPT(X1, Y1, Z1, 1000))
print("M2 permutation ", CPT(X2, Y2, Z2, 1000))
print("M3 permutation ", CPT(X3, Y3, Z3, 1000))

# asymptotic test for CMI
print("M1 asymptotic ", ACI(X1, Y1, Z1))
print("M2 asymptotic ", ACI(X2, Y2, Z2))
print("M3 asymptotic ", ACI(X3, Y3, Z3))

M1 permutation  (0.0006968093630406963, 0.4645354645354645)
M2 permutation  (0.0008915689687091646, 0.38461538461538464)
M3 permutation  (0.01031234541755329, 0.000999000999000999)
M1 asymptotic  (1.3936187260813926, 0.4981722575655283)
M2 asymptotic  (1.7831379374183292, 0.41001195190979545)
M3 asymptotic  (20.624690835106577, 3.322043306619449e-05)
